Преобразование признаков

In [20]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

Генерация признаков
* OHE для категориальных
* Возраст в годах
* Возраст группа  = гр_возраст      
* гр_возраст*пол = менешь 25 35 45 55 65 >75
* ИМТ
* ИМТ_гр = меньше 25, [25,30), [30,35),[35,40),>40
* разница арт давление (разница больше 60)
* Повышенное давление от группы 
http://dlyaserdca.ru/davlenie/lechenie-vysokogo-davleniya-pri-normalnom-nizhnem.html
* Артериальная гипертензия (
http://sosudinfo.ru/arterii-i-veny/arterialnaya-gipertenziya/

In [47]:
# Преобразовываем категориальные признаки в OHE
def ftr_categ_ohe(X,cols):
    def ohe_col(X,col_name):
        dum_df=pd.get_dummies(X[col_name])
        dum_df=dum_df.add_prefix(col_name)
        return X.join(dum_df).drop(labels=col_name,axis=1)
    for col in cols:
        X=ohe_col(X,col)
    return X
def ftr_age_from_days(data):
    data.age=data.age/365
    return data
def ftr_skip(data):
    return data.replace(to_replace='None',value=-1,inplace=True)
# кг / м^2
def ftr_BMI(data):
    data['BMI']=data['weight']/((data['height']/100)**2)

In [54]:
# Основноый метод преобразования и добавления признаков 
def feature_eng(data):
    # Удаляем id
    cat_drop=['id']
    data.drop(labels=cat_drop,axis=1,inplace=True)
    # OHE для категориальных
    #cat_cols=['gender','cholesterol','gluc']
    #cat_cols=['gender']
    #data=ftr_categ_ohe(data,cat_cols)
    # Преобразовываем возраст 
    data=ftr_age_from_days(data)
    
    # Индекс массы тела
    ftr_BMI(data)
    
    # Убираем пропуски 
    data.replace(to_replace='None',value=0,inplace=True)
    #data=ftr_skip(data,'smoke')
    #data=ftr_skip(data,'alco')
    #data=ftr_skip(data,'active')
    return data

In [55]:
all_train=pd.read_csv('train.csv',sep=';')
all_test=pd.read_csv('test.csv',sep=';')
all_train=feature_eng(all_train)
all_test=feature_eng(all_test)
all_test.head(20)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,BMI
0,51.747945,1,154,85.0,130,80,1,1,0,0,1,35.840783
1,52.169863,2,170,69.0,130,90,1,1,0,0,1,23.875433
2,55.978082,1,160,70.0,120,75,1,1,0,0,0,27.343750
3,49.679452,2,185,94.0,130,90,1,1,0,0,1,27.465303
4,44.090411,2,170,76.0,120,80,1,1,0,0,1,26.297578
5,45.786301,1,160,68.0,120,80,1,1,0,0,0,26.562500
6,43.487671,2,170,68.0,120,80,1,1,0,0,0,23.529412
7,55.956164,1,164,60.0,120,80,1,1,0,0,0,22.308150
8,55.512329,1,170,68.0,110,70,1,1,0,0,1,23.529412
9,59.553425,1,162,88.0,180,100,2,2,0,1,1,33.531474


In [56]:
all_train.to_csv('pre_train.csv',sep=';',index='index')
all_test.to_csv('pre_test.csv',sep=';',index='index')

In [8]:
all_train=pd.read_csv('train.csv',sep=';')
all_test=pd.read_csv('test.csv',sep=';')
cat_drop=['id']
all_train.drop(labels=cat_drop,axis=1,inplace=True)
all_test.drop(labels=cat_drop,axis=1,inplace=True)
all_train.to_csv('pre_train.csv',sep=';',index='index')
all_test.to_csv('pre_test.csv',sep=';',index='index')

### Борьба с пропусками
* Чистка данных, есть бредовые значения, их заменяем пропусками
* Средним по полу
* Предсказывать с помощью машинного обучения по признакам